In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

In [ ]:
# Stochastic Lie group dynamics

In [ ]:
import src.params as params
params.group = 'SON'
params.manifold = 'S2'

In [ ]:
from src.group import *
from src.manifold import *
from src.metric import *

from src.plotting import *
%matplotlib inline
plt.rcParams['figure.figsize'] = 7, 7

In [ ]:
# initialize
init(3)

In [ ]:
# setup for testing different versions of stochastic dynamics
q0 = np.array([1e-6,0,0])
g0 = psif(q0)
v0 = np.array([0,1,-1])
p0 = sharppsif(q0,v0)
mu0 = sharpVf(v0)

In [ ]:
# Brownian motion
from src.Brownian_inv import *

# srng.seed(422)
srng.seed(422)
(ts,gsv) = Brownian_invf(g0,dWsGf())
newfig()
plotg(gsv)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
plotM()
x0 = np.array([0,0,1])
plotx(actsf(gsv.transpose((1,2,0)),x0).T)
plt.show()
#plt.savefig('stocso32.pdf')

In [ ]:
# Brownian processes
from src.Brownian_process import *

(ts,gsv) = Brownian_processf(g0,dWsGf())
newfig()
plotg(gsv)
plt.show()

In [ ]:
# Stochastic coadjoint motion
from src.stochastic_coadjoint import *

(ts,musv) = stochastic_coadjointf(mu0,dWsGf())
(ts,gsv) = stochastic_coadjointrecf(g0,musv)
newfig()
plotg(gsv)
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.guided_process import *

# q0 = np.array([1e-6,0,1])
# g0 = psif(q0)
# v0 = q0+np.array([.5,0,0])

# phi = lambda g: -(g-v)
phi = lambda g,v: T.tensordot(inv(sigma),LAtoV(log(invtrns(inv(g),v))),(1,0))
sde_Brownian_inv_guided = get_sde_guided(sde_Brownian_inv,phi,lambda q: sigma)
Brownian_inv_guided = lambda g,v,dWt: integrate_sde(sde_Brownian_inv_guided,
                                                   integrator_stratonovich,
                                                   g,dWt,T.constant(0.),T.constant(0.),v)
v = T.matrix() # target
Brownian_inv_guidedf = theano.function([g,v,dWt], Brownian_inv_guided(g,v,dWt)[:4])

# derivatives
theta = sigma # parameters
def dlog_likelihood(g,v,dWt):
    s = Brownian_inv_guided(g,v,dWt)
    dlog_likelihood = T.grad(s[2][-1],theta)
    
    return [s[0],s[1],s[2],s[3],dlog_likelihood]
dlog_likelihoodf = theano.function([g,v,dWt],dlog_likelihood(g,v,dWt))

v = psif(v0)
(ts,gsv,log_likelihood,log_varphi) = Brownian_inv_guidedf(g0,v,dWsGf())
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])
newfig()
plotg(gsv)
plotg(v,color='k')
plt.show()

# on S2
newfig()
plotM()
plotx(actsf(gsv.transpose((1,2,0)),x0).T)
plotx(actf(v,x0),color='k')
plt.show()